In [15]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# -----------------------------------
# Merge the Datasets
# -----------------------------------

# Load the Steam and HLTB datasets
steam_games = pd.read_csv("/content/drive/My Drive/DSA210/project/steam_store_games.csv")  # Steam dataset
howlongtobeat_data = pd.read_csv("/content/drive/My Drive/DSA210/project/howlongtobeat_data.csv")  # HLTB dataset

# Check the datasets
print("First 5 rows of the Steam dataset:")
print(steam_games.head())
print("\nFirst 5 rows of the HLTB dataset:")
print(howlongtobeat_data.head())

# Merge the datasets using 'name' from Steam and 'title' from HLTB
merged_df = pd.merge(steam_games, howlongtobeat_data, how='inner', left_on='name', right_on='title')

# Check the merged dataset
print("\nFirst 5 rows of the merged dataset:")
print(merged_df.head())

# Save the merged dataset
merged_df.to_csv('/content/drive/My Drive/DSA210/project/merged_data.csv', index=False)
print("\nMerged dataset saved as 'merged_data.csv'.")

# -----------------------------------
# Clean the Merged Dataset
# -----------------------------------

# Drop unnecessary columns
columns_to_drop = [
    'appid', 'english', 'developer', 'publisher', 'genres_x',
    'achievements', 'id', 'title', 'type', 'genres_y',
    'release_na', 'release_eu', 'release_jp', 'categories',
    'coop', 'versus', 'publishers',
    'platforms_x', 'platforms_y'
]
merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

# Rename 'steamspy_tags' to 'genres'
merged_df = merged_df.rename(columns={'steamspy_tags': 'genres'})

# Convert time columns to numeric
time_columns = ['average_playtime', 'median_playtime', 'main_story', 'main_plus_extras', 'completionist', 'all_styles']
for col in time_columns:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Convert Steam playtime columns to hours and round to 2 decimals
merged_df['average_playtime'] = (merged_df['average_playtime'] / 60).round(2)
merged_df['median_playtime'] = (merged_df['median_playtime'] / 60).round(2)

# Round HLTB time columns to 2 decimals
hltb_time_columns = ['main_story', 'main_plus_extras', 'completionist', 'all_styles']
for col in hltb_time_columns:
    if col in merged_df.columns:
        merged_df[col] = merged_df[col].round(2)

# Check for missing values
print("\nMissing Values in Each Column:")
print(merged_df.isnull().sum())
print("\nPercentage of Missing Values in Each Column:")
print(merged_df.isnull().sum() / len(merged_df) * 100)

# Drop columns with more than 50% missing values
missing_percentage = merged_df.isnull().sum() / len(merged_df) * 100
columns_to_drop_high_missing = missing_percentage[missing_percentage > 50].index
merged_df = merged_df.drop(columns=columns_to_drop_high_missing)
print("\nColumns Dropped Due to High Missing Values (>50%):")
print(columns_to_drop_high_missing.tolist())

# Update time_columns after dropping high-missing columns
time_columns = [col for col in time_columns if col in merged_df.columns]

# Rename 'all_styles' to 'average_completion_time'
if 'all_styles' in merged_df.columns:
    merged_df = merged_df.rename(columns={'all_styles': 'average_completion_time'})
    # Update time_columns to reflect the new column name
    time_columns = ['average_playtime', 'median_playtime', 'average_completion_time']

# Drop rows with missing values in remaining time columns
merged_df = merged_df.dropna(subset=time_columns)

# Check for missing values after dropping rows
print("\nMissing Values After Dropping Rows:")
print(merged_df.isnull().sum())

# Check the dataset
print("\nUpdated Dataset - First 5 Rows:")
print(merged_df.head())
print("\nDataset Shape:", merged_df.shape)
print("\nColumns in the Dataset:", merged_df.columns.tolist())

# Save the cleaned dataset
merged_df.to_csv('/content/drive/My Drive/DSA210/project/clean_merged_data.csv', index=False, float_format='%.2f')
print("\nUpdated dataset saved as 'clean_merged_data.csv'.")

Mounted at /content/drive
First 5 rows of the Steam dataset:
   appid                       name release_date  english         developer  \
0     10             Counter-Strike   2000-11-01        1             Valve   
1     20      Team Fortress Classic   1999-04-01        1             Valve   
2     30              Day of Defeat   2003-05-01        1             Valve   
3     40         Deathmatch Classic   2001-06-01        1             Valve   
4     50  Half-Life: Opposing Force   1999-11-01        1  Gearbox Software   

  publisher          platforms  required_age  \
0     Valve  windows;mac;linux             0   
1     Valve  windows;mac;linux             0   
2     Valve  windows;mac;linux             0   
3     Valve  windows;mac;linux             0   
4     Valve  windows;mac;linux             0   

                                          categories  genres  \
0  Multi-player;Online Multi-Player;Local Multi-P...  Action   
1  Multi-player;Online Multi-Player;Local Multi